## SnowPit Data - XML data to python dataframe
#### Alexis Emerson

In [66]:
import xmltodict
import pandas as pd

In [67]:
file_path = "../raw_data/sp/all-MT.xml"
with open(file_path, "rb") as file:
    pit_data = xmltodict.parse(file)

In [68]:
# pit_data

In [69]:
df = pd.DataFrame.from_dict(pit_data['Pit_Data']['Pit_Observation'])
df.columns = [col.replace('@', '') for col in df.columns]

In [70]:
pd.set_option('display.max_columns', None)
df.head()

,activities,iLayerNumber,iDepth,bld,nid,stability,aviPit,crownObs,aviLoc,incline,winDir,skiAreaPit,bcPit,testPit,windspeed,aspect,sky,windLoading,heightOfSnowpack,precip,serial,version,pitNotes,timestamp,measureFrom,depthUnits,coordType,elvUnits,longType,latType,longitude,lat,zone,north,east,rhoUnits,fractureCat,hardnessScaling,state,range,share,User,Location,Layer,Shear_Test_Result,Density_Profile,Temperature_Profile,bootPen,skiPen,surfGrainType,surfGrainSize,tempUnits
0,,4,65,7.91,45338,Good,false,false,,17,E,false,,false,Moderate,90,CLR,yes,110,NO,drupal-nid-45338-1670119867000,7.91-0.1,very dense wind slabs throughout snowpack. Ang...,1670099400000,bottom,cm,lat_long,ft,W,N,-110.9561110,45.5247470,12T,5041247,503427,kg_cubic_m,shear_quality,linear,MT,Gallatin Range-N,true,"{'@measureFrom': 'bottom', '@depthUnits': 'cm'...","{'@ID': '', '@type': 'LATLONG', '@zone': '12T'...","[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","[{'@code': 'CT', '@sdepth': '65.0', '@score': ...","{'@profile': '', '@densityUnits': 'kg_cubic_m'...","{'@temp_profile': '', '@depthUnits': 'cm', '@t...",NaN,NaN,NaN,NaN,NaN
1,Ski tracks on slope; We skied slope;,2,64,7.91,45310,Good,false,false,,,SW,false,,false,Light Breeze,90,CLR,previous,75,NO,drupal-nid-45310-1670108769000,7.91-0.1,,1670097600000,bottom,cm,lat_long,ft,W,N,-111.0128870,45.4818490,12T,5036480,498993,kg_cubic_m,shear_quality,linear,MT,Gallatin Range-N,true,"{'@measureFrom': 'bottom', '@depthUnits': 'cm'...","{'@ID': '', '@type': 'LATLONG', '@zone': '12T'...","[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","[{'@code': 'ECT', '@sdepth': '20.0', '@score':...","{'@profile': '', '@densityUnits': 'kg_cubic_m'...","{'@temp_profile': '', '@depthUnits': 'cm', '@t...",NaN,NaN,NaN,NaN,NaN
2,Recent avalanche activity on similar slopes; R...,6,50,7.91,45286,Poor,false,false,,25,SW,false,,false,Light Breeze,230,BKN,previous,132,S-1,drupal-nid-45286-1670041124000,7.91-0.1,Recent natural avalanches suspected to have br...,1670020200000,bottom,cm,lat_long,ft,W,N,-109.9444300,45.0449700,,,,kg_cubic_m,shear_quality,linear,MT,Cooke City,true,"{'@measureFrom': 'bottom', '@depthUnits': 'cm'...","{'@ID': '', '@type': 'LATLONG', '@zone': '', '...","[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","[{'@code': 'ECT', '@sdepth': '64.0', '@score':...","{'@profile': '', '@densityUnits': 'kg_cubic_m'...","{'@temp_profile': '', '@depthUnits': 'cm', '@t...",NaN,NaN,NaN,NaN,NaN
3,,6,22,7.91,45347,,false,false,,10,S,false,,false,Light Breeze,190,FEW,previous,125,NO,drupal-nid-45347-1670123989000,7.91-0.1,,1670016600000,bottom,cm,lat_long,ft,W,N,-111.31703,44.96578,,,,kg_cubic_m,shear_quality,linear,MT,Madison Range-S,true,"{'@measureFrom': 'bottom', '@depthUnits': 'cm'...","{'@ID': '', '@type': 'LATLONG', '@zone': '', '...","[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","[{'@code': 'ECT', '@sdepth': '98.0', '@score':...","{'@profile': '', '@densityUnits': 'kg_cubic_m'...","{'@temp_profile': '', '@depthUnits': 'cm', '@t...",NaN,NaN,NaN,NaN,NaN
4,,6,22,7.91,45346,,false,false,,10,S,false,,false,Light Breeze,190,FEW,previous,125,NO,drupal-nid-45346-1670123988000,7.91-0.1,,1670016600000,bottom,cm,lat_long,ft,W,N,-111.3170300,44.9657800,,,,kg_cubic_m,shear_quality,linear,MT,Madison Range-S,true,"{'@measureFrom': 'bottom', '@depthUnits': 'cm'...","{'@ID': '', '@type': 'LATLONG', '@zone': '', '...","[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","[{'@code': 'ECT', '@sdepth': '98.0', '@score':...","{'@profile': '', '@densityUnits': 'kg_cubic_m'...","{'@temp_profile': '', '@depthUnits': 'cm', '@t...",NaN,NaN,NaN,NaN,NaN


https://www.statology.org/valueerror-if-using-all-scalar-values-you-must-pass-an-index/, https://www.geeksforgeeks.org/reset-index-in-pandas-dataframe/

## Flattening Columns

### Location Columns

In [71]:
locationdf = pd.DataFrame()

for i,loca in df.Location.items():
    l = pd.DataFrame([loca])
    locationdf = pd.concat([l, locationdf], axis='rows')
locationdf.reset_index(inplace = True)
locationdf.drop('index', axis=1, inplace = True)

### User Columns

In [72]:
userdf = pd.DataFrame()

for i,use in df.User.items():
    u = pd.DataFrame([use])
    userdf = pd.concat([u, userdf], axis='rows')
userdf.reset_index(inplace = True)
userdf.drop('index', axis=1, inplace = True)

### Concatenate into a new DataFrame

In [78]:
df2 = pd.concat([df, locationdf, userdf], axis='columns')
df2.columns = [col.replace('@', '') for col in df2.columns]

## Set Index to 'nid'

In [79]:
df3 = df2.set_index('nid')

## New Layers Dataframe Building

In [76]:
dflayers = pd.DataFrame()

for i,layers in df3.Layer.items():
    if isinstance(layers, dict):
        layers = [layers]
    if isinstance(layers, list):
        ph = pd.DataFrame(layers)
        ph['nid'] = i
        dflayers = pd.concat([dflayers, ph], axis='rows')

dflayers.columns = [col.replace('@', '') for col in dflayers.columns]
dflayers

,grainSizeUnits1,grainSizeUnits2,grainSize,grainSize1,grainType,grainType1,hardness1,hardness2,layerNumber,waterContent,startDepth,endDepth,multipleHardness,multipleGrainType,multipleGrainSize,nid
0,mm,mm,,,,,F,,1,D,110,100,false,false,false,45338
1,mm,mm,,,,,1F,,2,D,100,90,false,false,false,45338
2,mm,mm,,,,,4F-,,3,D,90,65,false,false,false,45338
3,mm,mm,,,,,4F,,4,D,65,35,false,false,false,45338
4,mm,mm,,,,,4F+,4F+,5,D,35,0,false,false,false,45338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,mm,mm,,,DF,,1F,,4,,141,126,false,false,false,32324
4,mm,mm,,,DF,,1F+,,5,,126,55,false,false,false,32324
5,mm,mm,1,,FCxr,,P,,6,,55,34,false,false,false,32324
6,mm,mm,2,4,FC,,4F,,7,,34,15,false,false,false,32324


## New Shear_Test_Results Dataframe Building

In [77]:
dfshear = pd.DataFrame()

for i,shear in df3.Shear_Test_Result.items():
    if isinstance(shear, dict):
        layers = [shear]
    if isinstance(shear, list):
        sh = pd.DataFrame(shear)
        sh['nid'] = i
        dfshear = pd.concat([dfshear, sh], axis='rows')

dfshear.columns = [col.replace('@', '') for col in dfshear.columns]
dfshear

,code,sdepth,score,ecScore,ctScore,quality,dateString,numberOfTaps,fractureCat,fractureCharacter,s,lengthOfCut,lengthOfColumn,releaseType,dataCode,nid,depthUnits
0,CT,65.0,CT,,21,Q2,12/03/2022,21,shear_quality,,CT21 Q2 65.0 12/03/2022.13:30:00:0,,,,,45338,NaN
1,ECT,65.0,ECTP,13,,,12/03/2022,13,shear_quality,,ECTP13 65.0 12/03/2022.13:30:00:0,,,,,45338,NaN
0,ECT,20.0,ECTN,29,,,12/03/2022,29,shear_quality,,ECTN29 20.0 12/03/2022.13:00:00:0,,,,,45310,NaN
1,ECT,64.0,ECTN,24,,,12/03/2022,24,shear_quality,,ECTN24 64.0 12/03/2022.13:00:00:0,,,,,45310,NaN
0,ECT,64.0,ECTN,23,,,12/02/2022,23,shear_quality,,ECTN23 64.0 12/02/2022.15:30:00:0,,,,,45286,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,ST,80.0,STH,,,,03/08/2021,,shear_quality,,STH 80.0 03/08/2021.11:00:00:0,,,,,32318,cm
1,CT,,CTN,,,,03/08/2021,,shear_quality,,CTN 03/08/2021.11:00:00:0,,,,,32318,cm
2,ECT,,ECTX,,,,03/08/2021,,shear_quality,,ECTX 03/08/2021.11:00:00:0,,,,,32318,cm
0,ECT,,ECTX,,,,03/08/2021,,shear_quality,,ECTX 03/08/2021.11:00:00:0,,,,,32324,cm
